# [Advent of Code 2020: Day 3](https://adventofcode.com/2019/day/3)

## \-\-\- Day 3: Toboggan Trajectory \-\-\-

With the toboggan login problems resolved, you set off toward the airport. While travel by toboggan might be easy, it's certainly not safe: there's very minimal steering and the area is covered in trees. You'll need to see which angles will take you near the fewest trees.

Due to the local geology, trees in this area only grow on exact integer coordinates in a grid. You make a map (your puzzle input) of the open squares (`.`) and trees (`#`) you can see. For example:

```
..##.......
#...#...#..
.#....#..#.
..#.#...#.#
.#...##..#.
..#.##.....
.#.#.#....#
.#........#
#.##...#...
#...##....#
.#..#...#.#

```

These aren't the only trees, though; due to something you read about once involving arboreal genetics and biome stability, the same pattern repeats to the right many times:

```
..##.........##.........##.........##.........##.........##.......  --->
#...#...#..#...#...#..#...#...#..#...#...#..#...#...#..#...#...#..
.#....#..#..#....#..#..#....#..#..#....#..#..#....#..#..#....#..#.
..#.#...#.#..#.#...#.#..#.#...#.#..#.#...#.#..#.#...#.#..#.#...#.#
.#...##..#..#...##..#..#...##..#..#...##..#..#...##..#..#...##..#.
..#.##.......#.##.......#.##.......#.##.......#.##.......#.##.....  --->
.#.#.#....#.#.#.#....#.#.#.#....#.#.#.#....#.#.#.#....#.#.#.#....#
.#........#.#........#.#........#.#........#.#........#.#........#
#.##...#...#.##...#...#.##...#...#.##...#...#.##...#...#.##...#...
#...##....##...##....##...##....##...##....##...##....##...##....#
.#..#...#.#.#..#...#.#.#..#...#.#.#..#...#.#.#..#...#.#.#..#...#.#  --->

```

You start on the open square (`.`) in the top\-left corner and need to reach the bottom (below the bottom\-most row on your map).

The toboggan can only follow a few specific slopes (you opted for a cheaper model that prefers rational numbers); start by *counting all the trees* you would encounter for the slope *right 3, down 1*:

From your starting position at the top\-left, check the position that is right 3 and down 1. Then, check the position that is right 3 and down 1 from there, and so on until you go past the bottom of the map.

The locations you'd check in the above example are marked here with `*O*` where there was an open square and `*X*` where there was a tree:

```
..##.........##.........##.........##.........##.........##.......  --->
#..O#...#..#...#...#..#...#...#..#...#...#..#...#...#..#...#...#..
.#....X..#..#....#..#..#....#..#..#....#..#..#....#..#..#....#..#.
..#.#...#O#..#.#...#.#..#.#...#.#..#.#...#.#..#.#...#.#..#.#...#.#
.#...##..#..X...##..#..#...##..#..#...##..#..#...##..#..#...##..#.
..#.##.......#.X#.......#.##.......#.##.......#.##.......#.##.....  --->
.#.#.#....#.#.#.#.O..#.#.#.#....#.#.#.#....#.#.#.#....#.#.#.#....#
.#........#.#........X.#........#.#........#.#........#.#........#
#.##...#...#.##...#...#.X#...#...#.##...#...#.##...#...#.##...#...
#...##....##...##....##...#X....##...##....##...##....##...##....#
.#..#...#.#.#..#...#.#.#..#...X.#.#..#...#.#.#..#...#.#.#..#...#.#  --->

```

In this example, traversing the map using this slope would cause you to encounter `*7*` trees.

Starting at the top\-left corner of your map and following a slope of right 3 and down 1, *how many trees would you encounter?*

In [1]:
import os
import unittest
import numpy as np
from operator import itemgetter
from fractions import Fraction

class TreeMap():
    
    EMPTY = '.'
    TREE = '#'
    NEWLINE = "\n"
    
    MAX_TIMES_OFF_GRID = 4
    
    def __init__(self, map_data = []):
        if map_data:
            self.load_data(map_data)
        else:
            self.data = map_data
        
        self.start_coord = (0,0)
        
    def load_map(self, path):
        with open(path) as file:
            raw_data = file.read()
        self.load_data(raw_data)
        
    def load_data(self, map_data):
        x = 0
        y = 0
        self.data = []
        for point in map_data:
            if point is self.TREE:
                self.data.append((x,y))
                
            if point is self.NEWLINE:
                y += 1
                x = 0
            else:
                x +=1
        
        self.width = x + 1
        self.height = y + 1
    
    def incriment_coord(self, coord, step):
        x,y = coord
        xstep, ystep = step
        return (x+xstep, y+ystep)
    
    def is_tree(self, coord):
        """ Does a tree exist at the given coordinate """
        x,y = coord
        
        # Since the tree pattern repeats on the x axis
        # we must ensure x will always be a valid value
        x = x % (self.width - 1)
        
        if y > self.height:
            raise Exception(f'Coordinate {coord} is off grid')
            
        return ((x,y) in self.data)
    
    def count_trees(self, slope):
        coord = self.start_coord
        count = 0
        while coord[1] < self.height:
            if coord[0] != 0:
                if self.is_tree(coord):
                    count += 1
            coord = self.incriment_coord(coord, slope)
        return count

class TestBasic(unittest.TestCase):
    
    def test_load_data(self):
        map_data = ".#..#\n.....\n#####\n....#\n...##"
        ans = [(1,0),(4,0),(0,2),(1,2),(2,2),(3,2),(4,2),(4,3),(3,4),(4,4)]
        tm = TreeMap(map_data)
        self.assertEqual(tm.data ,ans)
        
    def test_is_tree(self):
        map_data = ".#..#\n.....\n#####\n....#\n...##"
        test_data = [((1,0),True),((0,1),False),((6,0),True)]
        tm = TreeMap(map_data)
        for coord, ans in test_data:
            self.assertEqual(tm.is_tree(coord), ans)
            
    def test_count_trees(self):
        map_data = '..##.......\n#...#...#..\n.#....#..#.\n..#.#...#.#\n.#...##..#.\n..#.##.....\n.#.#.#....#\n.#........#\n#.##...#...\n#...##....#\n.#..#...#.#'
        ans = 7
        tm = TreeMap(map_data)
        self.assertEqual(tm.count_trees((3,1)),ans)
        
        
unittest.main(argv=[""], exit=False)

...
----------------------------------------------------------------------
Ran 3 tests in 0.003s

OK


In [2]:
tm = TreeMap()
tm.load_map("input/input_d03.txt")
tm.count_trees((3,1))

148

--- Part Two ---

Time to check the rest of the slopes - you need to minimize the probability of a sudden arboreal stop, after all.

Determine the number of trees you would encounter if, for each of the following slopes, you start at the top-left corner and traverse the map all the way to the bottom:

    Right 1, down 1.
    Right 3, down 1. (This is the slope you already checked.)
    Right 5, down 1.
    Right 7, down 1.
    Right 1, down 2.

In the above example, these slopes would find 2, 7, 3, 4, and 2 tree(s) respectively; multiplied together, these produce the answer 336.

What do you get if you multiply together the number of trees encountered on each of the listed slopes?

In [3]:
class TestBasic(unittest.TestCase):

    def test_count_trees(self):
        map_data = '..##.......\n#...#...#..\n.#....#..#.\n..#.#...#.#\n.#...##..#.\n..#.##.....\n.#.#.#....#\n.#........#\n#.##...#...\n#...##....#\n.#..#...#.#'
        test_data = [((1,1),2),((3,1),7),((5,1),3),((7,1),4),((1,2),2)]
        tm = TreeMap(map_data)
        for slope, ans in test_data:
            self.assertEqual(tm.count_trees(slope),ans)
        
        
unittest.main(argv=[""], exit=False)

.
----------------------------------------------------------------------
Ran 1 test in 0.001s

OK


In [4]:
tm = TreeMap()
tm.load_map("input/input_d03.txt")

slope_list = [(1,1),(3,1),(5,1),(7,1),(1,2)]

tree_count_list = []
for slope in slope_list:
    tree_count_list.append(tm.count_trees(slope))
    print(slope, tree_count_list[-1])
    
product = 1
for tree_count in tree_count_list:
    product *= tree_count
print(f'Tree count product: {product}')

(1, 1) 50
(3, 1) 148
(5, 1) 53
(7, 1) 64
(1, 2) 29
Tree count product: 727923200
